In [1]:
import psycopg2
from psycopg2.extras import DictCursor

In [2]:
def connect():
    return psycopg2.connect(dbname='postgres', user='postgres', password='coursework')

In [3]:
def get_items(conn, user_id):
    with conn.cursor(cursor_factory=DictCursor) as cursor:
        select = f"SELECT product, quantity FROM shopping_list WHERE user_id = '{user_id}';"
        cursor.execute(select)
        records = cursor.fetchall()
    return records

In [4]:
conn = connect()

In [5]:
records = list(zip(*get_items(conn, 1)))
try:
    shopping_list = list(records[0])
    quantity_list = list(records[1])
except:
    shopping_list = []
    quantity_list = []
shopping_list, quantity_list

(['bread', 'milk', 'eggs', 'tea'], [1, 1, 1, 1])

In [6]:
def add_items(conn, user_id, products, quantity):
    with conn.cursor(cursor_factory=DictCursor) as cursor:
        rows = list(zip(*(products, quantity)))
        VALUES = ', '.join(f"('{user_id}', '{row[0]}', {row[1]}, current_timestamp(0))" for row in rows)
        insert = f"""INSERT INTO 
                        shopping_list (user_id, product, quantity, created_on)
                     VALUES 
                        {VALUES}
                     ON CONFLICT ON CONSTRAINT shopping_list_user_id_product_key DO UPDATE
                        SET quantity = shopping_list.quantity + excluded.quantity,
                            created_on = current_timestamp(0);"""
        cursor.execute(insert)
        conn.commit()
    return

In [7]:
add_items(conn, 1, ['milk', 'bread', 'eggs', 'tea', 'noodles'], [2, 1, 3, 4, 1])

In [ ]:
def del_items(conn, user_id, products, quantity):
    with conn.cursor(cursor_factory=DictCursor) as cursor:
        rows = list(zip(*(products, quantity)))
        VALUES = ', '.join(f"('{user_id}', '{row[0]}', {row[1]}, current_timestamp(0))" for row in rows)
        insert = f"""INSERT INTO 
                        shopping_list (user_id, product, quantity, created_on)
                     VALUES 
                        {VALUES}
                     ON CONFLICT ON CONSTRAINT shopping_list_user_id_product_key DO UPDATE
                        SET quantity = shopping_list.quantity + excluded.quantity,
                            created_on = current_timestamp(0);"""
        cursor.execute(insert)
        conn.commit()
    return